In [ ]:
import os
from evaluation.save_results import create_output_file, save_results

output_file_name = "chronos_benchmark_results.csv"

output_dir="results"
csv_file_path = os.path.join(output_dir, output_file_name)

create_output_file(output_file_name, output_dir)


In [ ]:
from gluonts.model import evaluate_model
from gluonts.time_feature import get_seasonality
from evaluation.load_chronos_data import load_data

In [ ]:
from utevaluationils.metrics import get_metrics

# Instantiate the metrics
metrics = get_metrics()

In [ ]:
from evaluation.chronos_predictor import ChronosPredictor

for model_name, model_path in zip(CHRONOS_MODEL_NAME, CHRONOS_MODEL):
    for ds_name in DATASET_NAME:
        print(f"Processing dataset: {ds_name}")

        terms = ["short", "medium", "long"]
        for term in terms:
            ds_config = f"{ds_name}/{term}"

            dataset, prediction_length, frequency, domain, num_variates = load_data(ds_name, term)
            season_length = get_seasonality(frequency)

            predictor = ChronosPredictor(
                model_path=model_path,
                num_samples=20,
                prediction_length=prediction_length,
                # Change device_map to "cpu" to run on CPU or "cuda" to run on GPU
                device_map="cuda",
            )

            res = evaluate_model(
                predictor,
                test_data=dataset,
                metrics=metrics,
                batch_size=512,
                axis=None,
                mask_invalid_label=True,
                allow_nan_forecast=False,
                seasonality=season_length,
            )

            # Append the results to the CSV file
            save_results(res, ds_config, model_name, domain, num_variates, ds_name, csv_file_path)